In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

Chrome_options = Options()
Chrome_options.add_argument("--verbose")
Chrome_options.add_argument('--no-sandbox')
Chrome_options.add_argument("--headless=new")
Chrome_options.add_argument("--disable-gpu")
Chrome_options.add_argument("--windows-size=1920,1200")
Chrome_options.add_argument("--disable-dev-shm-usage")
Chrome_options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=Chrome_options)

/Users/hoof/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
def safe_find_element(driver, by, value):
    try:
        return driver.find_element(by, value)
    except NoSuchElementException:
        return None

def news_scraping(news_url, driver):
    # 언론사
    press_element = safe_find_element(driver, By.XPATH, '//*[@id="ct"]/div[1]/div[1]/a/img[2]')
    press = press_element.get_attribute('title') if press_element else ""

    # 기사 제목
    title_element = safe_find_element(driver, By.ID, 'title_area')
    title = title_element.text if title_element else ""

    # 발행일자
    date_time_element = safe_find_element(driver, By.XPATH, '//*[@id="ct"]/div[1]/div[3]/div[1]/div/span')
    date_time = date_time_element.text if date_time_element else ""

    # 기자
    repoter_element = safe_find_element(driver, By.XPATH, '//*[@id="JOURNALIST_CARD_LIST"]/div[1]/div/div[1]/div/div/div[1]/a[2]/span/span/em')
    repoter = repoter_element.text if repoter_element else ""

    # 기사 본문
    article_element = safe_find_element(driver, By.ID, 'dic_area')
    article = article_element.text.replace("\n", "").replace("\t", "") if article_element else ""

    # 기사 반응: 쏠쏠정보
    useful_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[1]/a/span[2]')
    useful = useful_element.text if useful_element else ""

    # 기사 반응: 흥미진진
    wow_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[2]/a/span[2]')
    wow = wow_element.text if wow_element else ""

    # 기사 반응: 공감백배
    touched_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[3]/a/span[2]')
    touched = touched_element.text if touched_element else ""

    # 기사 반응: 분석탁월
    analytical_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[4]/a/span[2]')
    analytical = analytical_element.text if analytical_element else ""

    # 기사 반응: 후속강추
    recommend_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[5]/a/span[2]')
    recommend = recommend_element.text if recommend_element else ""

    print("뉴스:", [title, press, date_time, repoter, article, useful, wow, touched, analytical, recommend, news_url])

    return [title, press, date_time, repoter, article, useful, wow, touched, analytical, recommend, news_url]

def scraping(list_url):
    driver.implicitly_wait(3)

    news_idx = 1
    news_df = pd.DataFrame(columns = ("Title", "Press", "DateTime", "Repoter", "Article", "Useful", "Wow", "Touched", "Analytical", "Recommend", "URL"))

    for url in list_url:
        driver.get(url)
        news_df.loc[news_idx] = news_scraping(url, driver)
        news_idx += 1

    driver.close()

    return news_df

In [3]:
def make_pg_num(num):
    """Calculate the page number in the format required by the website."""
    return num if num == 1 else num+9*(num-1)

def create_url(search, page_num):
    """Create a URL with the search term and page number."""
    return f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search}&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=17&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start={page_num}"

def make_urls(search, start_pg, end_pg):
    """Generate the URLs for the range of pages."""
    return [create_url(search, make_pg_num(i)) for i in range(start_pg, end_pg+1)]

def input_with_validation(prompt):
    """Ask for input with the given prompt, repeating until a valid integer is provided."""
    while True:
        try:
            return int(input(prompt))
        except ValueError:
            print("Invalid input, please enter an integer.")

def main():
    search = input("검색 키워드를 입력해주세요: ")

    start_pg = input_with_validation("\n크롤링 시작 페이지를 입력해주세요. ex)1(숫자만 입력): ")
    print(f"\n크롤링 시작 페이지: {start_pg}페이지")

    end_pg = input_with_validation("\n크롤링 종료 페이지를 입력해주세요. ex)1(숫자만 입력): ")
    print(f"\n크롤링 종료 페이지: {end_pg}페이지")

    return make_urls(search, start_pg, end_pg)

if __name__ == "__main__":
    search_urls = main()
    print("생성된 URL: ", search_urls)


크롤링 시작 페이지: 1페이지

크롤링 종료 페이지: 2페이지
생성된 URL:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=전세자금대출&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=17&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=전세자금대출&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=17&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start=11']


In [4]:
# Initialize the list to store the links
list_url = []

# Iterate over the URLs
for url in search_urls:
    # Send GET request to the web page
    response = requests.get(url)

    # If the request is successful, extract the HTML content and create a BeautifulSoup object
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        links = soup.select("a.info, a.sub_txt")

        # Filter and save the links with "naver.com" in their address
        for link in links:
            href = link.get("href")
            if "naver.com" in href:
                list_url.append(href)
    else:
        print("The request failed.")

    # Sleep for 1 second
    time.sleep(2)

In [5]:
list_url

['https://n.news.naver.com/mnews/article/001/0014024476?sid=101',
 'https://n.news.naver.com/mnews/article/018/0005516361?sid=101',
 'https://n.news.naver.com/mnews/article/082/0001218855?sid=101',
 'https://n.news.naver.com/mnews/article/001/0014024471?sid=101',
 'https://n.news.naver.com/mnews/article/001/0014024472?sid=101',
 'https://n.news.naver.com/mnews/article/003/0011932949?sid=101',
 'https://n.news.naver.com/mnews/article/003/0011933598?sid=101',
 'https://n.news.naver.com/mnews/article/047/0002396694?sid=100',
 'https://n.news.naver.com/mnews/article/024/0000082857?sid=101',
 'https://n.news.naver.com/mnews/article/001/0014022627?sid=101',
 'https://n.news.naver.com/mnews/article/215/0001108170?sid=101',
 'https://n.news.naver.com/mnews/article/015/0004857255?sid=101',
 'https://n.news.naver.com/mnews/article/014/0005028550?sid=101',
 'https://n.news.naver.com/mnews/article/018/0005511540?sid=101',
 'https://n.news.naver.com/mnews/article/421/0006872146?sid=101',
 'https://

In [6]:
scraping(list_url)

뉴스: ['가계대출 증가세', '연합뉴스', '2023.06.25. 오후 3:00', '류영석 기자', '(서울=연합뉴스) 류영석 기자 = 한국은행의 통화 긴축 기조에도 가계대출이 계속해 증가하는 가운데 25일 오후 서울의 한 시중은행에 가계 대출 상품 관련 현수막이 걸려 있다.이날 금융권에 따르면 5대 은행(KB국민·신한·하나·우리·NH농협)의 22일 기준 가계대출 잔액은 678조2천162억원으로 5월 말(677조6천122억원)보다 6천40억원 불어난 상태다. 세부적으로는 전세자금대출을 포함한 주택담보대출(잔액 510조1천596억원)이 22일까지 4천834억원 늘었다. 2023.6.25ondol@yna.co.kr', '0', '0', '0', '0', '0', 'https://n.news.naver.com/mnews/article/001/0014024476?sid=101']
뉴스: ['대출금리 다시 올랐는데…6월에도 은행 가계대출 상승세', '이데일리', '2023.06.25. 오후 2:07', '정두리 기자', '22일 기준 5대은행 가계대출 5월말보다 6050억 증가주담대는 4천834억↑…신용대출도 8개월 만 반등한은 내부선 가계대출 반등 조짐 경고 계속 나와사진=연합뉴스[이데일리 정두리 기자] 연초대비 대출금리가 크게 낮아지면서 가계대출 증가세가 이어질 조짐이다. 최근 시장금리의 상승으로 대출금리가 소폭 상승세지만 이달 들어서도 5대 은행에서만 가계대출이 6000억원 이상 늘었다.25일 금융권에 따르면 5대 은행(KB국민·신한·하나·우리·NH농협)의 22일 기준 가계대출 잔액은 678조2162억원으로 5월 말(677조6122억원)보다 6040억원 증가했다.지난달 5대 은행 가계대출은 전월인 4월(677조4691억원)보다 1431억원 늘어 1년 5개월만에 처음 전월대비 증가한 바 있다. 이달 말까지 남은 영업일을 감안하면 두달째 증가가 사실상 확살시된다. 증가 폭도 눈에 띄게 커질 전망이다.세부적으로는 전세자금대출을 포함한 주택담보대출(잔액 510조159

,Title,Press,DateTime,Repoter,Article,Useful,Wow,Touched,Analytical,Recommend,URL
1,가계대출 증가세,연합뉴스,2023.06.25. 오후 3:00,류영석 기자,(서울=연합뉴스) 류영석 기자 = 한국은행의 통화 긴축 기조에도 가계대출이 계속해 ...,0,0,0,0,0,https://n.news.naver.com/mnews/article/001/001...
2,대출금리 다시 올랐는데…6월에도 은행 가계대출 상승세,이데일리,2023.06.25. 오후 2:07,정두리 기자,22일 기준 5대은행 가계대출 5월말보다 6050억 증가주담대는 4천834억↑…신용...,0,0,0,0,0,https://n.news.naver.com/mnews/article/018/000...
3,이달 5대 은행 가계대출 6000억 넘게 늘어…증가세 이어질 듯,부산일보,2023.06.25. 오후 1:31,권기택 기자,금리 올랐지만 주담대 4천834억원↑…신용대출도 8개 월 만에 반등국내 5대 은행의...,1,0,0,0,0,https://n.news.naver.com/mnews/article/082/000...
4,가계대출 증가세,연합뉴스,2023.06.25. 오후 3:00,류영석 기자,(서울=연합뉴스) 류영석 기자 = 한국은행의 통화 긴축 기조에도 가계대출이 계속해 ...,0,0,0,0,0,https://n.news.naver.com/mnews/article/001/001...
5,가계대출 증가세,연합뉴스,2023.06.25. 오후 3:00,류영석 기자,(서울=연합뉴스) 류영석 기자 = 한국은행의 통화 긴축 기조에도 가계대출이 계속해 ...,0,0,0,0,0,https://n.news.naver.com/mnews/article/001/001...
6,"""이자 내 줄테니 더 살아줘요""...세입자 모시는 집주인[역전세난 시대]①",뉴시스,2023.06.24. 오전 6:01,박성환 기자,"전셋값 급락으로 역전세난에 이어 역월세까지1년 내 만기 전세보증금 300조원…""역대...",0,7,19,6,8,https://n.news.naver.com/mnews/article/003/001...
7,"""갭투자 했던 게 화근""…집주인들 고통의 시간[역전세난 시대]②",뉴시스,2023.06.25. 오전 6:01,강세훈 기자,"""제 때 돈 돌려달라"" 내용증명 독촉 사례 속출전세금 돌려줄 돈 없어 고민 깊은 갭...",17,8,20,4,7,https://n.news.naver.com/mnews/article/003/001...
8,"원희룡 장관님, 전세사기 원인 그게 정말 맞나요",오마이뉴스,2023.06.23. 오후 2:24,,"[주장] ""더불어민주당 정권"" 탓이라는 발언의 문제점... 대책에 국고지원 필요한 ...",8,0,0,0,0,https://n.news.naver.com/mnews/article/047/000...
9,“6억짜리가 3억으로”...전세 ‘반 토막’에 송도 집주인 ‘곡소리’ [김경민의 부...,매경이코노미,2023.06.23. 오후 8:46,김경민 기자,송도 더샵센트럴시티 전용 59㎡ 3억 하락전셋값 급락에 수도권 역전세난 심화 우려인...,9,16,55,4,10,https://n.news.naver.com/mnews/article/024/000...
10,"""부동산, 작년 말 저점 지났다…강남권은 전세 하방 압력↑""",연합뉴스,2023.06.23. 오후 6:00,민선희 기자,"김경민 교수, 정책포럼서 발표…""2025년 이후 아파트 입주 물량 급감""부동산[연합...",3,0,5,0,1,https://n.news.naver.com/mnews/article/001/001...
